In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('dark_background')

from scipy.optimize import root
from scipy.integrate import solve_ivp
from numpy.linalg import eig

In [9]:
def numerical_jacobian(f, g, x0, h=1e-5):
    x, y = x0

    df_dx = (f(x + h, y) - f(x - h, y)) / (2 * h)
    df_dy = (f(x, y + h) - f(x, y - h)) / (2 * h)
    dg_dx = (g(x + h, y) - g(x - h, y)) / (2 * h)
    dg_dy = (g(x, y + h) - g(x, y - h)) / (2 * h)

    return np.array([[df_dx, df_dy], [dg_dx, dg_dy]])

def find_fixed_points(x_range, y_range, f, g, resolution=30, tol=1e-5):
    x_vals = np.linspace(*x_range, resolution)
    y_vals = np.linspace(*y_range, resolution)
    fixed_points = []

    for x in x_vals:
        for y in y_vals:
            sol = root(lambda v: [f(v[0], v[1]), g(v[0], v[1])], [x, y])
            if sol.success:
                pt = tuple(np.round(sol.x, decimals=5))
                if pt not in fixed_points:
                    fixed_points.append(pt)
    return fixed_points

def plot_phase_portrait(x_range, y_range, f, g, density=20, stream=False):
    x = np.linspace(*x_range, density)
    y = np.linspace(*y_range, density)
    X, Y = np.meshgrid(x, y)
    U, V = f(X,Y), g(X,Y)
    fixed_points = find_fixed_points(x_range,y_range,f,g)

    fig, ax = plt.subplots(figsize=(8, 8))
    
    if stream:
        ax.streamplot(X, Y, U, V,color='lightgray')
    else:
        ax.quiver(X, Y, U, V, color='tab:blue')

    for point in fixed_points:
        x0, y0 = point
        J = numerical_jacobian(f, g, (x0, y0))
        eigvals, eigvecs = eig(J)

        for val, vec in zip(eigvals, eigvecs.T):
            direction = vec / np.linalg.norm(vec)
            real = np.real(val)
            tol = 1e-4
            color = 'tab:red' if real > tol else 'tab:green' if real < -tol else 'tab:olive'
            if np.imag(val) != 0:
                # Complex eigenvalues: draw blue circle
                circle = plt.Circle((x0, y0), np.imag(val), color=color, fill=False, alpha=0.7)
                ax.add_artist(circle)
            else:
                # Real eigenvalues: draw lines
                dx, dy = direction * 0.5
                ax.plot([x0 - dx, x0 + dx], [y0 - dy, y0 + dy], color=color, linewidth=2, alpha=0.7)

        ax.plot(x0, y0, 'wo')  # mark the fixed point

    ax.set_xlim(*x_range)
    ax.set_ylim(*y_range)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal')
    ax.grid(True, linestyle='--', color='darkgray', linewidth=0.8, zorder=0)
    return fig, ax

# IMAGES

In [ ]:
a,b,c,d = 0.55, 0.02, 0.008, 0.01
def xdot(x, y):
    return -(c + a*b*y)*x

def ydot(x, y):
    return -(d + (1-a)*b*x)*y

def system(t, z):
    x, y = z
    return [xdot(x, y), ydot(x, y)]

x_range = (-1, 4)
y_range = (-1, 4)

f = lambda x,y: xdot(x,y)
g = lambda x,y: ydot(x,y)
fig, ax = plot_phase_portrait(x_range, y_range, f, g, density=15,stream=True)


# Add a particular trajectory
t_span = (0, 45)
z0 = [2.7,2.7]  # initial condition
t_eval = np.linspace(*t_span, 1000)
sol = solve_ivp(system, t_span, z0, t_eval=t_eval)
ax.plot(sol.y[0], sol.y[1], color='red', linewidth=2, label='1980-2025 Population')
ax.legend()

ax.set_title(f'Model I Population Trajectory a={a}',fontsize=20)
ax.set_xlabel('Female',fontsize=18)
ax.set_ylabel('Male',fontsize=18)
fig.savefig(f"flows/china_model")
plt.show()
plt.close(fig)

TypeError: only integer scalar arrays can be converted to a scalar index

# VIDEO

In [13]:
!echo Y | del frames\* 

c:\Users\artka\Videos\MEME-DEEZ\frames\*, Are you sure (Y/N)? Y 


In [17]:
def xdot(x, y, c):
    return (x**2 - np.sign(c)*c**2)

def ydot(x, y, c):
    return y

x_range = (-3, 3)
y_range = (-3, 3)
c_range = np.linspace(-1,1,101)

for i, c in enumerate(c_range):
    f = lambda x,y: xdot(x,y,c)
    g = lambda x,y: ydot(x,y,c)
    fig, ax = plot_phase_portrait(x_range, y_range, f, g)
    ax.set_title(f'Saddle Node c = {c:.2f}')
    fig.savefig(f"frames/frame_{i:03d}")
    plt.close(fig)

In [18]:
!echo Y | ffmpeg -r 10 -i frames\frame_%03d.png -vcodec libx264 -pix_fmt yuv420p saddle_node.mp4


ffmpeg version 7.1.1-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-dxva2 --enable-d3d11va --enable-d3d12va --enable-ffnvcodec --enable-libvpl --enable-nvdec --enable-nvenc --enable-vaapi --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame -